In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_atendente"
TABELA_DESTINO = "v_credit.gold.dm_atendente"

In [0]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_atendente"),
    F.col("nm_atendente"),
    F.col("nu_nivel").cast("smallint"),
    
    F.when(
        F.col("nu_nivel") == 2, "Especialista (N2)"
    ).otherwise("Generalista (N1)").alias("ds_perfil"),
    
    F.current_timestamp().alias("dt_ingestion"),
    F.lit("ETL_Databricks_Merge").alias("dc_origem")
)

deltaTable = DeltaTable.forName(spark, TABELA_DESTINO)

deltaTable.alias("target").merge(
    df_gold.alias("source"),
    "target.cd_atendente = source.cd_atendente"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Merge (Upsert) realizado com sucesso em {TABELA_DESTINO}")